### Data Collection

**Objectives**
- Fetch data from Kaggle and prepare it for further processes

**Inputs**
- Kaggle JSON file - authentication token

**Outputs**
- Generate Dataset: inputs/datasets/codeinstitute/cherry-leaves

**Additional Comments | Insights | Conclusions**
- No comments

**Import Packages**

In [1]:
import numpy as np
import os




**Change the working directory**

In [2]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves/jupyter_notebooks'

In [3]:
os.chdir('/workspace/mildew-detection-in-cherry-leaves')
print("You set a new current directory")

You set a new current directory


In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves'

**Install Kaggle**

In [5]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 27.3 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.8-py3-none-any.whl size=111962 sha256=8f48e33c5aba97c82adfa4f953f4bed05d1ca16be1751372e218a9dfad29a1fe
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/90/a0/a1/c8af89a81821187aef4675b6e8c18f40a8cdbe8b00fd0e2d76
Successfully built kaggle


Change kaggle configuration directory to current working directory and permission of kaggle authentication json

In [7]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set Kaggle Dataset and Download it

In [11]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaves_dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 91%|██████████████████████████████████▌   | 50.0M/55.0M [00:01<00:00, 42.1MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 36.7MB/s]


Unzip the downloaded file, delete the zip file

In [14]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')